## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [1]:
import numpy as np

N = 1000000

# Генерация массива A из N случайных целых чисел на отрезке от 0 до 1000
A = np.random.randint(0, 1001, N)

# Вычисление массива B
B = A + 100

# Вычисление среднего значения массива B
mean_B = np.mean(B)

print("Среднее значение массива B:", mean_B)


Среднее значение массива B: 599.86536


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [4]:
import pandas as pd
import numpy as np
import string

# Создание таблицы с 2 млн строк и 4 столбцами, заполненными случайными числами
df = pd.DataFrame(np.random.randint(0, 100, size=(2000000, 4)), columns=list('ABCD'))

# Добавление столбца key, содержащего случайные английские буквы
df['key'] = np.random.choice(list(string.ascii_lowercase), 2000000)

# Выборка строк, где в столбце key указаны первые 5 английских букв
subset = df[df['key'].str[:5].isin(list(string.ascii_lowercase)[:5])]

subset


,A,B,C,D,key
3,24,78,2,12,c
13,54,43,10,8,a
20,66,48,29,6,e
21,69,64,5,44,e
27,95,95,14,6,e
...,...,...,...,...,...
1999979,42,43,36,64,c
1999985,32,96,29,29,a
1999986,0,68,25,86,c
1999989,7,63,79,51,b


## Лабораторная работа 3

In [5]:
!pip install line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.9/661.9 kB 9.9 MB/s eta 0:00:00


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [6]:
# Загрузка данных из файлов
recipes = pd.read_csv('recipes_sample.csv')
reviews = pd.read_csv('reviews_sample.csv')

# Приведение столбцов к нужным типам
recipes['submitted'] = pd.to_datetime(recipes['submitted'])
reviews['date'] = pd.to_datetime(reviews['date'])



In [7]:
def calculate_mean_rating_a(df):
    total_rating = 0
    count = 0
    
    for _, row in df.iterrows():
        if row['date'].year == 2010:
            total_rating += row['rating']
            count += 1
    
    if count == 0:
        return 0
    
    return total_rating / count

mean_rating_a = calculate_mean_rating_a(reviews)
print("Среднее значение рейтинга (метод A):", mean_rating_a)

Среднее значение рейтинга (метод A): 4.4544402182900615


In [8]:
%%time
calculate_mean_rating_a(reviews)

CPU times: user 5.78 s, sys: 21.1 ms, total: 5.8 s
Wall time: 5.86 s


4.4544402182900615

In [9]:
def calculate_mean_rating_b(df):
    total_rating = 0
    count = 0
    
    for _, row in df[df['date'].dt.year == 2010].iterrows():
        total_rating += row['rating']
        count += 1
    
    if count == 0:
        return 0
    
    return total_rating / count

mean_rating_b = calculate_mean_rating_b(reviews)
print("Среднее значение рейтинга (метод B):", mean_rating_b)


Среднее значение рейтинга (метод B): 4.4544402182900615


In [10]:
%%time
calculate_mean_rating_b(reviews)

CPU times: user 594 ms, sys: 1.91 ms, total: 595 ms
Wall time: 597 ms


4.4544402182900615

In [11]:
def calculate_mean_rating_c(df):
    mean_rating = df[df['date'].dt.year == 2010]['rating'].mean()
    
    if pd.isnull(mean_rating):
        return 0
    
    return mean_rating

mean_rating_c = calculate_mean_rating_c(reviews)
print("Среднее значение рейтинга (метод C):", mean_rating_c)


Среднее значение рейтинга (метод C): 4.4544402182900615


In [12]:
%%time
calculate_mean_rating_c(reviews)

CPU times: user 22.1 ms, sys: 0 ns, total: 22.1 ms
Wall time: 22.3 ms


4.4544402182900615

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [13]:
import line_profiler

# Создание экземпляра профайлера
profiler = line_profiler.LineProfiler(calculate_mean_rating_a, calculate_mean_rating_b, calculate_mean_rating_c)

# Запуск профайлера
profiler.enable()

# Выполнение функций
mean_rating_a = calculate_mean_rating_a(reviews)
mean_rating_b = calculate_mean_rating_b(reviews)
mean_rating_c = calculate_mean_rating_c(reviews)

# Отключение профайлера и вывод результатов
profiler.disable()
profiler.print_stats()


Timer unit: 1e-09 s

Total time: 0.0189009 s
File: <ipython-input-11-13f1430a9322>
Function: calculate_mean_rating_c at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def calculate_mean_rating_c(df):
     2         1   18893943.0 18893943.0    100.0      mean_rating = df[df['date'].dt.year == 2010]['rating'].mean()
     3                                               
     4         1       6760.0   6760.0      0.0      if pd.isnull(mean_rating):
     5                                                   return 0
     6                                               
     7         1        186.0    186.0      0.0      return mean_rating

Total time: 11.422 s
File: <ipython-input-7-e660e5f5c04b>
Function: calculate_mean_rating_a at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def calculate_mean_rating_a(df):
     2         1       1710.0   171

Методы B и C, которые предварительно фильтруют таблицу, могут быть более эффективными по сравнению с методом A, который выполняет операции для всех строк.

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [14]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [15]:
import cProfile

def profile_get_word_reviews_count():
    cProfile.run('get_word_reviews_count(reviews)', sort='tottime')

profile_get_word_reviews_count()


         52986037 function calls (51972523 primitive calls) in 42.593 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    9.595    9.595   42.480   42.480 <ipython-input-14-b1bc049bcd0c>:1(get_word_reviews_count)
   253364    2.383    0.000   21.466    0.000 series.py:342(__init__)
10846018/9832522    1.694    0.000    2.072    0.000 {built-in method builtins.len}
8995349/8995343    1.625    0.000    2.411    0.000 {built-in method builtins.isinstance}
   253364    1.369    0.000    6.066    0.000 construction.py:493(sanitize_array)
   253372    1.244    0.000    1.834    0.000 generic.py:5844(__finalize__)
  6792020    1.073    0.000    1.073    0.000 {method 'append' of 'list' objects}
   380037    1.069    0.000    5.112    0.000 series.py:966(__getitem__)
   506720    1.064    0.000    1.064    0.000 {method 'split' of 'str' objects}
   253358    0.968    0.000    1.673    0.000 cast.py:1178(maybe_infer_to_date

In [16]:
from collections import defaultdict

def get_word_reviews_count_optimized(df):
    word_reviews_count = defaultdict(int)
    
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        
        unique_words = set(words)  # Уникальные слова в отзыве
        
        for word in unique_words:
            word_reviews_count[word] += 1
    
    return word_reviews_count


In [23]:
import line_profiler

# Создание экземпляра профайлера
profiler = line_profiler.LineProfiler(get_word_reviews_count_optimized, get_word_reviews_count)

# Запуск профайлера
profiler.enable()

# Выполнение функций
word_reviews_count_optimized = get_word_reviews_count_optimized(reviews)
word_reviews_count = get_word_reviews_count(reviews)

# Отключение профайлера и вывод результатов
profiler.disable()
profiler.print_stats()


Timer unit: 1e-09 s

Total time: 49.9161 s
File: <ipython-input-14-b1bc049bcd0c>
Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def get_word_reviews_count(df):
     2         1       2134.0   2134.0      0.0      word_reviews = {}
     3    126679 11517114845.0  90915.7     23.1      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679 2649829252.0  20917.7      5.3          recipe_id, review = row['recipe_id'], row['review']
     5    126679  626643221.0   4946.7      1.3          words = review.split(' ')
     6   6792010 1626330327.0    239.4      3.3          for word in words:
     7   6617066 3244072996.0    490.3      6.5              if word not in word_reviews:
     8    174944   96768705.0    553.1      0.2                  word_reviews[word] = []
     9   6792010 4114151667.0    605.7      8.2              word_reviews[word].append(recipe_id)
    10

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [32]:
%%time
def calculate_mape_v1(df):
    total_mape = 0
    count = 0
    
    for _, row in df.iterrows():
        rating = row['rating']
        if rating != 0:
            average_rating = df[df['recipe_id'] == row['recipe_id']]['rating'].mean()
            mape = abs(rating - average_rating) / average_rating
            total_mape += mape
            count += 1
    
    if count > 0:
        return total_mape / count
    else:
        return None


CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 9.54 µs


In [35]:
%%time
from numba import njit

@njit
def calculate_mape_v2(df):
    total_mape = 0
    count = 0
    
    for _, row in df.iterrows():
        rating = row['rating']
        if rating != 0:
            average_rating = df[df['recipe_id'] == row['recipe_id']]['rating'].mean()
            mape = abs(rating - average_rating) / average_rating
            total_mape += mape
            count += 1
    
    if count > 0:
        return total_mape / count
    else:
        return None


CPU times: user 353 µs, sys: 0 ns, total: 353 µs
Wall time: 361 µs


In [34]:
%%time
import numpy as np

def calculate_mape_v3(df):
    ratings = df['rating'].values
    recipe_ids = df['recipe_id'].values
    
    mask = (ratings != 0)
    non_zero_ratings = ratings[mask]
    non_zero_recipe_ids = recipe_ids[mask]
    
    average_ratings = np.zeros_like(non_zero_ratings)
    unique_recipe_ids = np.unique(non_zero_recipe_ids)
    
    for recipe_id in unique_recipe_ids:
        mask = (non_zero_recipe_ids == recipe_id)
        average_ratings[mask] = np.mean(non_zero_ratings[mask])
    
    mape = np.abs(non_zero_ratings - average_ratings) / average_ratings
    
    return np.mean(mape)


CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 13.1 µs


In [36]:
%%time
from numba import njit

@njit
def calculate_mape_v4(df):
    ratings = df['rating'].values
    recipe_ids = df['recipe_id'].values
    
    mask = (ratings != 0)
    non_zero_ratings = ratings[mask]
    non_zero_recipe_ids = recipe_ids[mask]
    
    average_ratings = np.zeros_like(non_zero_ratings)
    unique_recipe_ids = np.unique(non_zero_recipe_ids)
    
    for recipe_id in unique_recipe_ids:
        mask = (non_zero_recipe_ids == recipe_id)
        average_ratings[mask] = np.mean(non_zero_ratings[mask])
    
    mape = np.abs(non_zero_ratings - average_ratings) / average_ratings
    
    return np.mean(mape)


CPU times: user 336 µs, sys: 0 ns, total: 336 µs
Wall time: 343 µs
